##Install libs

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

##Drive for train and val datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Config

In [ ]:
base_model_name = "speakleash/Bielik-4.5B-v3.0-Instruct"
load_from_checkpoint = False
checkpoint_dir = None # Not applicable since load_from_checkpoint is False

max_seq_length = 32768
MAX_NEW_TOKENS = 256
BATCH_SIZE = 2
load_in_4bit = True

##Load model

In [ ]:
from google.colab import userdata

hf_token = userdata.get('HF_API_KEY')

In [ ]:
from unsloth import FastLanguageModel

if load_from_checkpoint:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,
        disable_log_stats = True,
    )
else:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = base_model_name,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,
        token = hf_token,
        disable_log_stats = True,
    )

2026-01-04 11:40:37,525 - modelscope - INFO - Target directory already exists, skipping creation.


==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


TimeoutError: Unsloth: HuggingFace seems to be down after trying for 120 seconds :(
Check https://status.huggingface.co/ for more details.
As a temporary measure, use modelscope with the same model name ie:
```
pip install modelscope
import os; os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'
from unsloth import FastLanguageModel
model = FastLanguageModel.from_pretrained('unsloth/gpt-oss-20b')
```

###Add LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

NameError: name 'model' is not defined

##Load datasets

In [ ]:
from datasets import load_dataset

train_data_path = "drive/MyDrive/mipd_train.jsonl"
val_data_path = "drive/MyDrive/mipd_val.jsonl"

# Load the datasets
datasets = load_dataset("json", data_files={
    "train": train_data_path,
    "validation": val_data_path
})

print(datasets)

train = datasets["train"]
val = datasets["validation"]

In [ ]:
train[101:103]

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # or "qwen-2.5"
)

# 2. Define the Function for Alpaca Format
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    # Iterate over the 3 columns simultaneously
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Convert your 3 columns into the standard list of messages
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": input},
            {"role": "assistant", "content": output}
        ]

        # Apply the template to turn the list into a string
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)

    return { "text" : texts }

# 3. Apply it
train_dataset = train.map(formatting_prompts_func, batched=True)
val_dataset = val.map(formatting_prompts_func, batched=True)

In [ ]:
train_dataset[101]['text']

##Init trainer

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[101]["input_ids"])

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

##Run sft

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
message = val[0]

messages = [
    {"role" : "system", "content" : message["instruction"]},
    {"role" : "user", "content" : message["input"]}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

print("--------")
print("Correct label: ", message["output"])